# Getting Started

In this tutorial, you will know how to
- use the models in **tatk** to build a dialog agent.
- build a simulator to chat with the agent.
- try different module combinations.

Let's get started!

In [ ]:
# first install tatk and restart the notebook
! git clone https://github.com/thu-coai/tatk.git && cd tatk && pip install -e .

## Build an agent

We use the models adapted on [Multiwoz dataset](https://www.aclweb.org/anthology/D18-1547) to build our agent. This pipeline agent consists of NLU, DST, Policy and NLG modules.

First, import some models:

In [1]:
import sys
import os
# Agent
from tatk.dialog_agent import PipelineAgent, BiSession
# common import: tatk.$module.$model.$dataset
from tatk.nlu.svm.multiwoz import SVMNLU
from tatk.nlu.bert.multiwoz import BERTNLU
from tatk.dst.rule.multiwoz import RuleDST
from tatk.policy.rule.multiwoz import Rule
from tatk.nlg.template.multiwoz import TemplateNLG

Then, create the models and build an agent:

In [2]:
# svm nlu trained on usr sentence of multiwoz
# go to README.md under `tatk/tatk/nlu/svm/multiwoz` for more information 
nlu = SVMNLU('usr',model_file="https://tatk-data.s3-ap-northeast-1.amazonaws.com/svm_multiwoz_usr.zip")
# simple rule DST
dst = RuleDST()
# rule policy
policy = Rule(character='sys')
# template NLG
nlg = TemplateNLG(is_user=False)
# assemble
sys_agent = PipelineAgent(nlu, dst, policy, nlg)

loading saved Classifier
/home/zhuqi/Codes/tatk/tatk/nlu/svm/multiwoz/model/svm_multiwoz_usr.pickle
loaded.


That's all! Let's chat with the agent using its `response` function:

In [3]:
sys_agent.response("I want to find a moderate hotel")

'There are 18 of those . How about acorn guest house ? fits your request perfectly .'

In [4]:
sys_agent.response("Which type of hotel is it ?")

'It is a guesthouse .'

In [5]:
sys_agent.response("OK , where is its address ?")

'They are located at pool way, whitehill road, off newmarket road .'

In [6]:
sys_agent.response("Thank you !")

'Welcome , it was a pleasure serving you .'

In [7]:
sys_agent.response("Try to find me a Chinese restaurant in south area .")

'I have 3 different restaurants i can give you some information for . they are all pretty good . Peking restaurant matches your description . They are located at 10 homerton street city centre.'

In [8]:
sys_agent.response("Which kind of food it provides ?")

'They serve chinese .'

In [9]:
sys_agent.response("Book a table for 5 , this Sunday .")

'Reference number is : 00000003 .'

## Build a Simulator to Chat with the Agent

In many one-to-one task-oriented dialog system, a simulator is essential to train an RL agent. In our framework, we doesn't distinguish user or system, all speakers are **agents**. The simulator is also an agent, with specific policy inside for accomplishing the user goal.

We use Agenda policy for the simulator, this policy requires dialog act input, which means we should set DST argument of `PipelineAgent` to `None`. Then the `PipelineAgent` will pass dialog act to policy directly. Refer to `PipelineAgent` doc for more details.

In [11]:
# bert nlu trained on sys sentence of multiwoz
# go to README.md under `tatk/tatk/nlu/bert/multiwoz` for more information 
nlu = BERTNLU('sys',model_file="https://tatk-data.s3-ap-northeast-1.amazonaws.com/bert_multiwoz_sys.zip")
# not use dst
dst = None
# rule policy
policy = Rule(character='usr')
# template NLG
nlg = TemplateNLG(is_user=True)
# assemble
simulator = PipelineAgent(nlu, dst, policy, nlg)

Load from /home/zhuqi/Codes/tatk/tatk/nlu/bert/multiwoz/output/sys/bestcheckpoint.tar
train step 18600
BERTNLU loaded
Loading goal model is done


Now we have a simulator and an agent. we will use an existed simple one-to-one conversation controller `BiSession`, you can also define your own Session class for your special need.

In [12]:
sess = BiSession(sys_agent, simulator, None)

Let's make this two agents chat! The key is `next_turn` method of Session class.

In [13]:
sys_response = 'null'
sess.init_session()
for i in range(30):
    sys_response, user_response, session_over, reward = sess.next_turn(sys_response)
    print('user:', user_response)
    print('sys:', sys_response)
    print('reward:', reward)
    if session_over is True:
        break

user: I need to travel on thursday .
sys: What time do you want to depart ? Is there a time you need to arrive by ? Where are you heading to ? Where will you be traveling from ?
reward: -1.0
user: I need a train going to cambridge . The train should leave on thursday . I need it to depart from cambridge .
sys: When would you like the train to depart ?
reward: -1.0
user: I 'd like to arrive by 17:15 .
sys: The car arriving will be a honda . The contact number is 24pw58dkk65 .
reward: -1.0
user: Can you tell me the price ? Could you comfirm the departure time of that train ?
sys: I would say you should leave by 05:00. It would cost 23.60 pounds .
reward: -1.0
user: What is the departure time ?
sys: I would say you should leave by 05:00.
reward: -1.0
user: I want to get there by 17:15 at the latest .
sys: Reference number is : 00000000 .
reward: -1.0
user: What is the departure time ?
sys: I have a train leaving at 05:00 would that be okay ?
reward: -1.0
user: Also , thanks for the apolog

## Try Different Module Combinations

The combination modes of pipeline agent modules are flexible. We support joint model such as [MDBT](https://www.aclweb.org/anthology/P18-2069) (NLU+DST) and [MDRG](https://pdfs.semanticscholar.org/47d0/1eb59cd37d16201fcae964bd1d2b49cfb55e.pdf) (Policy+NLG), once the input and output are matched with previous and next module. We also support End2End model such as [Sequicity](https://www.comp.nus.edu.sg/~kanmy/papers/acl18-sequicity.pdf).

### MDBT
- NLU: None
- DST: MDBT
- Policy: Rule
- NLG: TemplateNLG

In [ ]:
from tatk.dst.mdbt.multiwoz.mdbt import MultiWozMDBT
nlu = None
# simple rule DST
dst = MultiWozMDBT()
# rule policy
policy = Rule()
# template NLG
nlg = TemplateNLG(is_user=False)
# assemble
sys_agent = PipelineAgent(nlu, dst, policy, nlg)

### MDRG

- NLU: SVM
- DST: Rule
- Policy: MDRG
- NLG: None

In [ ]:
from tatk.policy.mdrg.multiwoz.policy import MDRGWordPolicy
# svm nlu trained on usr sentence of multiwoz
# go to README.md under `tatk/tatk/nlu/svm/multiwoz` for more information 
nlu = SVMNLU('usr',model_file="https://tatk-data.s3-ap-northeast-1.amazonaws.com/svm_multiwoz_usr.zip")
# simple rule DST
dst = RuleDST()
# rule policy
policy = MDRGWordPolicy()
# template NLG
nlg = None
# assemble
sys_agent = PipelineAgent(nlu, dst, policy, nlg)

### Sequicity

Sequicity inherits from interface `Agent` directly.

In [ ]:
from tatk.e2e.sequicity.multiwoz import Sequicity
sequicity = Sequicity()
sys_agent = sequicity